In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow_addons.metrics import RSquare


In [2]:
df_2022 = pd.read_csv('data/model_2022-2023.csv')

In [ ]:
df_2022.head()

In [3]:
df_2022.drop('Unnamed: 0', axis='columns',inplace=True)
df_2022.head()

,minutes/game,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_shotAttempts,I_F_points,I_F_rebounds,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_unblockedShotAttempts
0,9.32,49.47,6.70,3.80,79,15,5,15,16,33,29,7,1.16,3.58,1.96,0,4,1,69
1,15.01,37.10,1.36,1.83,78,14,0,27,20,50,2,0,1.09,0.28,0.00,1,1,0,52
2,13.04,69.79,2.44,3.85,140,5,4,9,18,97,5,0,1.98,0.46,0.00,2,0,0,102
3,14.02,32.17,1.21,1.81,79,8,5,8,12,46,1,0,1.13,0.08,0.00,1,1,0,47
4,16.00,39.69,6.81,2.85,65,16,5,6,12,29,14,10,1.07,1.73,4.02,1,2,4,53


In [4]:
df_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774 entries, 0 to 773
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   minutes/game               774 non-null    float64
 1   I_F_xOnGoal                774 non-null    float64
 2   I_F_xGoals                 774 non-null    float64
 3   I_F_xRebounds              774 non-null    float64
 4   I_F_shotAttempts           774 non-null    int64  
 5   I_F_points                 774 non-null    int64  
 6   I_F_rebounds               774 non-null    int64  
 7   I_F_takeaways              774 non-null    int64  
 8   I_F_giveaways              774 non-null    int64  
 9   I_F_lowDangerShots         774 non-null    int64  
 10  I_F_mediumDangerShots      774 non-null    int64  
 11  I_F_highDangerShots        774 non-null    int64  
 12  I_F_lowDangerxGoals        774 non-null    float64
 13  I_F_mediumDangerxGoals     774 non-null    float64

# Preparing the Data

In [5]:
#Splitting the data
x = df_2022.copy()
x.drop('I_F_points', axis=1, inplace=True)
X = x.values
y = df_2022['I_F_points']

In [6]:
#further splitting into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
#creating the scaler variable
scaler = StandardScaler()

In [8]:
X_scaler = scaler.fit(X_train)

In [9]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Linear Regression Model

In [10]:
linear_model = LinearRegression()

In [11]:
# Fit the data into the model
linear_model.fit(X_train_scaled, y_train)

LinearRegression()

In [12]:
# Display the model's best fit line formula
print(f"Model's formula: y = {linear_model.intercept_} + {linear_model.coef_[0]}X")

Model's formula: y = 28.337931034482736 + 1.8397295428445215X


In [13]:
#Make predictions with the model using X_test_scaled
predicted_y_values = linear_model.predict(X_test_scaled)


In [14]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = linear_model.score(X_test_scaled, y_test, sample_weight=None)
r2 = r2_score(y_test, predicted_y_values)
mse = mean_squared_error(y_test, predicted_y_values)
mae = mean_absolute_error(y_test, predicted_y_values)

rmse = np.sqrt(mse)
std = np.std(y_test)

# Print relevant metrics.
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The mean absolute error is {mae}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The r2 is 0.9355372094887995.
The mean squared error is 42.96367226632804.
The mean absolute error is 4.728647629304027.
The root mean squared error is 6.554667975292726.
The standard deviation is 25.81642765917697.


# Random Forest

In [15]:
# Create a random forest classifier
rf_model = RandomForestRegressor(n_estimators=1000, random_state=42)

In [16]:
# Fitting the model
rf_model_fit = rf_model.fit(X_train_scaled, y_train)

In [17]:
# Making predictions using the testing data
predictions = rf_model_fit.predict(X_test_scaled)

In [18]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = rf_model_fit.score(X_test_scaled, y_test, sample_weight=None)
r2 = r2_score(y_test, predicted_y_values)
mse = mean_squared_error(y_test, predicted_y_values)
mae = mean_absolute_error(y_test, predicted_y_values)

rmse = np.sqrt(mse)
std = np.std(y_test)

# Print relevant metrics.
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The mean absolute error is {mae}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The r2 is 0.9355372094887995.
The mean squared error is 42.96367226632804.
The mean absolute error is 4.728647629304027.
The root mean squared error is 6.554667975292726.
The standard deviation is 25.81642765917697.


# Neural Network

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="selu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="selu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="selu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="selu"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 95        
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 5)                 30        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="MeanSquaredError", optimizer="adam", metrics=[RSquare()])

In [21]:
# Train the model
# fit_model = nn.fit(X_train_scaled, y_train, epochs=525)
fit_model = nn.fit(X_train, y_train, epochs=525)

Epoch 1/525
19/19 [==============================] - 3s 5ms/step - loss: 1454.5702 - r_square: -1.6377
Epoch 2/525
19/19 [==============================] - 0s 4ms/step - loss: 1271.2927 - r_square: -1.3054
Epoch 3/525
19/19 [==============================] - 0s 4ms/step - loss: 1209.8284 - r_square: -1.1939
Epoch 4/525
19/19 [==============================] - 0s 4ms/step - loss: 1197.0370 - r_square: -1.1707
Epoch 5/525
19/19 [==============================] - 0s 3ms/step - loss: 1185.2081 - r_square: -1.1493
Epoch 6/525
19/19 [==============================] - 0s 4ms/step - loss: 1174.1598 - r_square: -1.1292
Epoch 7/525
19/19 [==============================] - 0s 4ms/step - loss: 1163.6322 - r_square: -1.1102
Epoch 8/525
19/19 [==============================] - 0s 5ms/step - loss: 1153.3296 - r_square: -1.0915
Epoch 9/525
19/19 [==============================] - 0s 5ms/step - loss: 1143.3422 - r_square: -1.0734
Epoch 10/525
19/19 [==============================] - 0s 5ms/step - loss:

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, R-Square: {model_accuracy}")

7/7 - 0s - loss: 110.1618 - r_square: 0.8347 - 341ms/epoch - 49ms/step
Loss: 110.16175842285156, R-Square: 0.8347130417823792


In [23]:
# Save the model
nn.save("points_predictor_nn.h5")